In [2]:
!pip install finance-datareader

In [50]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [62]:
# 삼성전자 주가 데이터 가져오기 (최근 51일 + 이동평균선을 계산하기 위해 사용한 20일)
stock_data = fdr.DataReader('005930')
stock_data = stock_data.iloc[-32:]
stock_data

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2023-06-27,72500,72600,72000,72600,9442997,0.002762
2023-06-28,72600,72700,72000,72700,8783093,0.001377
2023-06-29,73100,73400,72400,72400,12229967,-0.004127
2023-06-30,72500,72700,71700,72200,11694765,-0.002762
2023-07-03,72700,73200,72600,73000,10722181,0.011080
2023-07-04,73400,73600,72900,73000,10214350,0.000000
2023-07-05,73000,73300,71900,72000,12310610,-0.013699
2023-07-06,71900,72400,71500,71600,14777667,-0.005556
2023-07-07,71100,71400,69800,69900,17308877,-0.023743


In [66]:
# 이미지 크기 설정
width, height = 96, 96

# High, Low, Close, Volume 데이터 정규화
high_prices = stock_data['High'].values
low_prices = stock_data['Low'].values
close_prices = stock_data['Close'].values
volume_data = stock_data['Volume'].values


high_prices_norm = (high_prices - np.min(low_prices)) / (np.max(high_prices) - np.min(low_prices))
low_prices_norm = (low_prices - np.min(low_prices)) / (np.max(high_prices) - np.min(low_prices))
close_prices_norm = (close_prices - np.min(low_prices)) / (np.max(high_prices) - np.min(low_prices))
volume_data_norm = height * (volume_data) / (2 * (np.max(volume_data)))


# 이동평균선 추가를 위한 stock_data 생성
stock_data_ma20 = stock_data.copy()
stock_data_ma20['MA20'] = stock_data_ma20['Close'].rolling(window=20).mean()

# 이동평균선 값을 정규화
stock_data_ma20['MA20_norm'] = (
        (stock_data_ma20['MA20'] - np.min(low_prices)) / (np.max(high_prices) - np.min(low_prices))
    )

In [72]:
def alpha_blend(color1, color2, alpha):
    return [
        int((color1[0] * alpha + color2[0] * (1 - alpha))),
        int((color1[1] * alpha + color2[1] * (1 - alpha))),
        int((color1[2] * alpha + color2[2] * (1 - alpha))),
        255,
    ]

# 96x96 이미지 생성
img = np.zeros((height, width, 4), dtype=np.uint8) # 4 channels for RGBA

for day in range(32):
    high_price = int(high_prices_norm[day] * (height - 1))
    low_price = int(low_prices_norm[day] * (height - 1))
    close_price = int(close_prices_norm[day] * (height - 1))
    open_price = int(
        (
            (stock_data['Open'].values[day] - np.min(low_prices))
            / (np.max(high_prices) - np.min(low_prices))
        )
        * (height - 1)
    )
    volume = int(volume_data_norm[day])

    x_start = day * 3

    # 시가, 종가, 고가, 저가 막대그래프
    # 막대 왼쪽, 오른쪽 그리기 (시가, 종가 범위)
    for x in range(x_start, x_start + 3):
        if close_prices[day] > stock_data['Open'].values[day]:
            color = [255, 0, 0, 255]  # 상승: 빨강
        else:
            color = [0, 0, 255, 255]  # 하락: 파랑
        for y in range(min(open_price, close_price), max(open_price, close_price) + 1):
            img[height - 1 - y, x] = color


    # 꼬리 그리기 (High, Low 범위)
    for x in range(x_start + 1, x_start + 2):
        for y in range(low_price, high_price + 1):
            if img[height - 1 - y, x][0] != 255 and img[height - 1 - y, x][2] != 255:
                img[height - 1 - y, x] = [200, 200, 200, 255]

    # 거래량 그리기
    for x in range(x_start, x_start + 3):
        overlapped = height - volume
        if overlapped >= 0 and overlapped < height:
            img[overlapped:, x] = [
                alpha_blend(img[y, x], [128, 128, 128, 255], 0.45)
                for y in range(overlapped, height)
            ]

    # 20일 이동평균선 그리기
    ma20_norm = stock_data_ma20.iloc[day]['MA20_norm']

    for x in range(x_start, x_start + 3):
        if not np.isnan(ma20_norm):
            ma20_y = int(ma20_norm * (height - 1))
            if 0 <= ma20_y < height:
                for y in range(ma20_y - 1, ma20_y + 2):  # 이동평균선 주변에 블렌딩 처리
                    if 0 <= y < height:
                        new_color = alpha_blend([255, 255, 0, 255], img[height - 1 - y, x], 0.6)  # 노란색으로 변경
                        img[height - 1 - y, x] = new_color

# 이미지 저장
img = Image.fromarray(img, 'RGBA')
img.save('samsung_stock_data_image.png')
img.show()